In [2]:
import pandas as pd
import numpy as np

In [40]:
allmodis = pd.read_csv(r'..\Data\Raw\archive_M6.csv').drop(['acq_time', 'satellite', 'instrument', 'version', 'daynight', 'type'], 1)
allviirs = pd.read_csv(r'..\Data\Raw\archive_V1.csv').drop(['acq_time', 'satellite', 'instrument', 'version', 'type'], 1)

In [9]:
allmodis.head()

,latitude,longitude,brightness,scan,track,acq_date,confidence,bright_t31,frp
0,-16.4439,143.5076,319.9,4.5,2.0,2000-11-01,59,300.6,62.8
1,-16.4447,143.5152,321.0,4.5,2.0,2000-11-01,62,300.5,71.6
2,-18.4459,144.8904,324.6,3.2,1.7,2000-11-01,74,303.1,77.4
3,-18.4401,144.8603,317.7,3.3,1.7,2000-11-01,61,303.2,38.5
4,-18.6804,145.5470,317.3,2.9,1.6,2000-11-01,54,299.4,32.8


In [5]:
#allmodis.to_csv(r'..\Data\Clean\carchive_M6.csv', index = False)
#allviirs.to_csv(r'..\Data\Clean\carchive_V1.csv', index = False)

In [4]:
rain = pd.read_csv(r'..\Data\Raw\rain.csv')

In [6]:
rain.head()

,Rain ID,Date/Time,Nearest town,State,Latitude,Longitude,Severity confidence description,Intense precipitation amount,Intense precipitation period,Total precipitation,Total precipitation duration,Water course,Damage total,Insurance payout,Number of deaths,Number of injuries,Unnamed: 16
0,989,2006-01-06 05:05:00,walcha,NSW,-30.93,151.72,Estimate of rainfall (eg. from filling of rece...,25.0,10.0,35,0,NaN,0.0,0.0,0.0,0.0,NaN
1,990,2006-01-07 01:00:00,rosebank,NSW,-28.64,153.41,Other measurement of rainfall.,0.0,0.0,505,0,NaN,0.0,0.0,0.0,0.0,NaN
2,991,2006-01-12 05:15:00,Walcha,NSW,-30.99,151.59,Other measurement of rainfall.,54.0,35.0,54,0,NaN,0.0,0.0,0.0,0.0,NaN
3,992,2006-01-15 05:20:00,Forbes,NSW,-33.39,148.01,Other measurement of rainfall.,53.0,60.0,53,0,NaN,0.0,0.0,0.0,0.0,NaN
4,993,2006-01-15 21:54:00,Goulburn,NSW,-34.75,149.73,Bureau measurement of rainfall.,85.0,240.0,85,0,NaN,0.0,0.0,0.0,0.0,NaN


In [8]:
rain.nunique()

Rain ID                            3466
Date/Time                          2483
Nearest town                       2149
State                                 7
Latitude                           2183
Longitude                          2133
Severity confidence description       4
Intense precipitation amount        402
Intense precipitation period         84
Total precipitation                 195
Total precipitation duration         18
Water course                        102
Damage total                          5
Insurance payout                      1
Number of deaths                      4
Number of injuries                    1
Unnamed: 16                           0
dtype: int64

In [10]:
import reverse_geocoder as rg
rg.search((rain.Latitude[0], rain.Longitude[0]))

Loading formatted geocoded file...


[OrderedDict([('lat', '-30.73333'),
              ('lon', '151.8'),
              ('name', 'Enmore'),
              ('admin1', 'New South Wales'),
              ('admin2', 'Armidale Dumaresq'),
              ('cc', 'AU')])]

In [11]:
rain.drop(['Unnamed: 16', 'Severity confidence description', 'Water course', 'Number of injuries', 'Number of deaths', 'Damage total', 'Damage total'], 1)

,Rain ID,Date/Time,Nearest town,State,Latitude,Longitude,Intense precipitation amount,Intense precipitation period,Total precipitation,Total precipitation duration,Insurance payout
0,989,2006-01-06 05:05:00,walcha,NSW,-30.9300,151.7200,25.0,10.0,35,0,0.0
1,990,2006-01-07 01:00:00,rosebank,NSW,-28.6400,153.4100,0.0,0.0,505,0,0.0
2,991,2006-01-12 05:15:00,Walcha,NSW,-30.9900,151.5900,54.0,35.0,54,0,0.0
3,992,2006-01-15 05:20:00,Forbes,NSW,-33.3900,148.0100,53.0,60.0,53,0,0.0
4,993,2006-01-15 21:54:00,Goulburn,NSW,-34.7500,149.7300,85.0,240.0,85,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3461,6564,2018-12-16 06:45:00,HOBART,TAS,-42.8820,147.3230,8.8,5.0,0,0,NaN
3462,6565,2018-12-21 06:00:00,DEDDINGTON,TAS,-41.6050,147.4140,12.4,15.0,26,24,NaN
3463,6566,2019-04-05 15:00:00,CLIFTON BEACH,TAS,-42.9900,147.5310,8.4,3.0,15,24,NaN
3464,6567,2019-05-01 00:00:00,SMITHTON,TAS,-40.8440,145.1200,10.4,5.0,54,24,NaN


In [12]:
rain.to_csv(r'..\Data\Clean\rain.csv', index = False)

In [14]:
wind = pd.read_csv(r'..\Data\Raw\wind.csv')

In [3]:
wind.head()

,Wind ID,Date/Time,Duration,Latitude,Longitude,Nearest town,State,Source,Max Gust speed,Max Gust direction,Max Mean Wind speed,Max Mean Wind direction,Path length,Path width,Path direction,Fujita scale,Damage total,Insurance payout,Number of deaths,Number of injuries
0,1376,2006-01-01 03:32:00,0,-36.88,149.24,Bombala,NSW,AWS,48,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1,1377,2006-01-01 04:44:00,0,-35.94,148.38,Cabramurra,NSW,AWS,63,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0
2,1378,2006-01-01 05:16:00,0,-36.23,149.12,Cooma,NSW,AWS,56,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0
3,1379,2006-01-06 04:30:00,0,-30.93,151.72,walcha,NSW,SPOTTER,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0
4,1380,2006-01-06 08:45:00,0,-28.85,153.05,Casino,NSW,AWS,54,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0


In [15]:
wind.nunique()

Wind ID                    2833
Date/Time                  2715
Duration                     61
Latitude                   1246
Longitude                  1258
Nearest town               1146
State                         7
Source                      461
Max Gust speed               76
Max Gust direction          267
Max Mean Wind speed          45
Max Mean Wind direction      46
Path length                   7
Path width                    7
Path direction                8
Fujita scale                  3
Damage total                  3
Insurance payout              1
Number of deaths              2
Number of injuries            2
Unnamed: 20                   0
dtype: int64

In [20]:
wind.drop(['Unnamed: 20', 'Number of injuries', 'Number of deaths', 'Insurance payout', 'Damage total'], 1)

,Wind ID,Date/Time,Duration,Latitude,Longitude,Nearest town,State,Source,Max Gust speed,Max Gust direction,Max Mean Wind speed,Max Mean Wind direction,Path length,Path width,Path direction,Fujita scale
0,1376,2006-01-01 03:32:00,0,-36.880,149.2400,Bombala,NSW,AWS,48,0,0,0,0,0,0,0
1,1377,2006-01-01 04:44:00,0,-35.940,148.3800,Cabramurra,NSW,AWS,63,0,0,0,0,0,0,0
2,1378,2006-01-01 05:16:00,0,-36.230,149.1200,Cooma,NSW,AWS,56,0,0,0,0,0,0,0
3,1379,2006-01-06 04:30:00,0,-30.930,151.7200,walcha,NSW,SPOTTER,0,0,0,0,0,0,0,0
4,1380,2006-01-06 08:45:00,0,-28.850,153.0500,Casino,NSW,AWS,54,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2828,8249,2019-04-09 05:05:00,0,-30.514,151.6660,Armidale,NSW,NaN,49,0,0,0,0,0,0,0
2829,8250,2016-11-21 09:00:00,0,-39.224,146.9850,HOGAN ISLAND,TAS,Hogan Island AWS,63,0,0,0,0,0,0,0
2830,8251,2019-03-24 14:00:00,0,-40.920,144.7000,MARRAWAH,TAS,Marrawah 9am Synop,60,300,0,0,0,0,0,0
2831,8252,2019-04-05 12:00:00,0,-42.154,145.3260,STRAHAN,TAS,Strahan,60,0,0,0,0,0,0,0


In [21]:
wind.to_csv(r'..\Data\Clean\wind.csv', index = False)

In [23]:
lightning = pd.read_csv(r'..\Data\Raw\lightning.csv')

In [36]:
lightning.head()

,Lightning ID,Date/Time,Latitude,Longitude,Nearest town,State,Source,Severity confidence description,Max current,Flash rate,Damage total,Insurance payout,Number of deaths,Number of injuries,Power line deaths
0,242,2006-03-01 00:00:00,-35.0180,117.8840,Albany,WA,Albany Extra,NaN,0,0,0.0,0.0,0.0,0.0,0.0
1,266,2009-10-05 04:00:00,-34.5067,150.7929,DAPTO,NSW,NaN,based only on physical evidence at the scene a...,0,0,NaN,NaN,NaN,NaN,NaN
2,267,2009-11-17 07:00:00,-28.6651,153.2845,THE CHANNON,NSW,Spotter,based only on physical evidence at the scene a...,0,0,NaN,NaN,NaN,NaN,NaN
3,269,2011-03-11 03:40:00,-31.1095,150.9147,SOUTH TAMWORTH,NSW,Storm Spotter,based on actual observation of event by other ...,0,0,NaN,NaN,NaN,NaN,NaN
4,270,2011-02-28 08:15:00,-31.9652,141.4513,BROKEN HILL,NSW,Storm Spotter,based on actual observation of event by other ...,0,0,NaN,NaN,NaN,NaN,NaN


In [24]:
lightning.nunique()

Lightning ID                       20
Date/Time                          19
Latitude                           18
Longitude                          18
Nearest town                       18
State                               5
Source                              7
Severity confidence description     3
Max current                         1
Flash rate                          3
Damage total                        1
Insurance payout                    1
Number of deaths                    1
Number of injuries                  1
Power line deaths                   1
Unnamed: 15                         0
dtype: int64

In [25]:
lightning['Number of deaths'].unique()

array([ 0., nan])

In [26]:
lightning.drop(['Unnamed: 15', 'Power line deaths', 'Number of injuries', 'Number of deaths', 'Insurance payout', 'Damage total', 'Flash rate', 'Max current', 'Severity confidence description'], 1)

,Lightning ID,Date/Time,Latitude,Longitude,Nearest town,State,Source
0,242,2006-03-01 00:00:00,-35.0180,117.8840,Albany,WA,Albany Extra
1,266,2009-10-05 04:00:00,-34.5067,150.7929,DAPTO,NSW,NaN
2,267,2009-11-17 07:00:00,-28.6651,153.2845,THE CHANNON,NSW,Spotter
3,269,2011-03-11 03:40:00,-31.1095,150.9147,SOUTH TAMWORTH,NSW,Storm Spotter
4,270,2011-02-28 08:15:00,-31.9652,141.4513,BROKEN HILL,NSW,Storm Spotter
5,271,2011-05-03 06:00:00,-27.4417,152.9444,THE GAP,QLD,NaN
6,272,2006-04-21 05:00:00,-28.0667,153.4333,MIAMI,QLD,NaN
7,273,2006-04-05 07:30:00,-27.9881,152.9958,BEAUDESERT,QLD,NaN
8,274,2010-11-14 17:00:00,-12.4626,130.8429,DARWIN CITY,NT,Darwin
9,322,2012-02-18 09:45:00,-33.9076,148.1223,GRENFELL,NSW,Spotter


In [27]:
lightning.to_csv(r'..\Data\Clean\lightning.csv', index = False)